1. Build Identity Block 

2. Build Convolutional Block

3. Build ResNet50

4. Build ResNet on CIFAR10

4. Train ResNet on CIFAR10

5. Generalize ResNet

In [45]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import (Input, Conv2D, Activation, 
                                     BatchNormalization, Add,
                                     ZeroPadding2D, MaxPooling2D,
                                     GlobalAveragePooling2D,
                                     Dense, Flatten)
from tensorflow.keras import Model

# 1. Build Identity Block

In [50]:
def identity_block(X, filters):
    F1, F2, F3 = filters

    shortcut = X

    # Conv 1x1
    X = Conv2D(filters=F1, kernel_size=1)(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)

    # Conv 3x3
    X = Conv2D(filters=F2, kernel_size=3, padding='same')(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)

    # Conv 1x1
    X = Conv2D(filters=F3, kernel_size=1)(X)
    X = BatchNormalization()(X)

    # Shortcut
    if F3 != F2:
        shortcut = Conv2D(filters=F3, kernel_size=1)(shortcut)
    X = Add()([shortcut, X])
    X = Activation('relu')(X)

    return X

In [51]:
# Test out function
input = Input(shape=(32, 32, 16))
output = identity_block(input, filters=[16, 16, 16])

model = Model(inputs=input, outputs=output)
model.summary()

Model: "model_15"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_20 (InputLayer)       [(None, 32, 32, 16)]         0         []                            
                                                                                                  
 conv2d_327 (Conv2D)         (None, 32, 32, 16)           272       ['input_20[0][0]']            
                                                                                                  
 batch_normalization_289 (B  (None, 32, 32, 16)           64        ['conv2d_327[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activation_344 (Activation  (None, 32, 32, 16)           0         ['batch_normalization_2

# 2. Build Convolutional Block

In [52]:
def convolutional_block(X, filters, s=2):
    F1, F2, F3 = filters

    shortcut = X 

    # Conv 1x1
    X = Conv2D(filters=F1, kernel_size=1)(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)

    # Conv 3x3
    X = Conv2D(filters=F2, kernel_size=3, strides=s, padding='same')(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)

    # Conv 1x1
    X = Conv2D(filters=F3, kernel_size=1)(X)
    X = BatchNormalization()(X)

    # Convolutional shortcut
    shortcut = Conv2D(filters=F3, kernel_size=1, strides=s)(shortcut)
    shortcut = BatchNormalization()(shortcut)

    # Shortcut
    X = Add()([shortcut, X])
    X = Activation('relu')(X)

    return X

In [53]:
input = Input(shape=(32, 32, 16))
output = convolutional_block(input, filters=[16, 16, 16])

model = Model(inputs=input, outputs=output)
model.summary()

Model: "model_16"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_21 (InputLayer)       [(None, 32, 32, 16)]         0         []                            
                                                                                                  
 conv2d_330 (Conv2D)         (None, 32, 32, 16)           272       ['input_21[0][0]']            
                                                                                                  
 batch_normalization_292 (B  (None, 32, 32, 16)           64        ['conv2d_330[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activation_347 (Activation  (None, 32, 32, 16)           0         ['batch_normalization_2

# 3. Build ResNet50

In [54]:
def ResNet50(input_shape=(224, 224, 3), num_classes=1000):
    input = Input(input_shape)

    # Zero-padding
    X = ZeroPadding2D((3, 3))(input)

    # Stage 01
    X = Conv2D(filters=64, kernel_size=7, strides=2, activation='relu')(X)
    X = BatchNormalization()(X)
    X = MaxPooling2D(3, strides=2, padding='same')(X)

    # Stage 02
    for _ in range(3):
        X = identity_block(X, filters=[64, 64, 256])
    
    # Stage 03
    X = convolutional_block(X, filters=[128, 128, 512], s=2)
    for _ in range(3):
        X = identity_block(X, filters=[128, 128, 512])
        
    # Stage 04
    X = convolutional_block(X, filters=[256, 256, 1024], s=2)
    for _ in range(5):
        X = identity_block(X, filters=[256, 256, 1024])

    # Stage 05
    X = convolutional_block(X, filters=[512, 512, 2048], s=2)
    for _ in range(2):
        X = identity_block(X, filters=[512, 512, 2048])

    X = GlobalAveragePooling2D()(X)

    # Ouput layer
    X = Flatten()(X)
    X = Dense(units=num_classes, activation='softmax')(X)

    # Create model
    model = Model(inputs=input, outputs=X)

    return model

In [55]:
model = ResNet50()
model.summary()

Model: "model_17"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_22 (InputLayer)       [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 zero_padding2d_10 (ZeroPad  (None, 230, 230, 3)          0         ['input_22[0][0]']            
 ding2D)                                                                                          
                                                                                                  
 conv2d_334 (Conv2D)         (None, 112, 112, 64)         9472      ['zero_padding2d_10[0][0]']   
                                                                                                  
 batch_normalization_296 (B  (None, 112, 112, 64)         256       ['conv2d_334[0][0]']   

# 4. Build ResNet on CIFAR10

In [58]:
def ResNet_Cifar10(input_shape=(32, 32, 3), num_classes=10):
    input = Input(input_shape)

    # Stage 1
    X = Conv2D(filters=64, kernel_size=3, padding='same')(input)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    # Stage 2
    for _ in range(3):
        X = identity_block(X, filters=[16, 16, 64])

    # Stage 3
    X = convolutional_block(X, filters=[32, 32, 128], s=2)
    for _ in range(6):
        X = identity_block(X, filters=[32, 32, 128])
    
    # Stage 4
    X = convolutional_block(X, filters=[64, 64, 256], s=2)
    for _ in range(3):
        X = identity_block(X, filters=[64, 64, 256])

    X = GlobalAveragePooling2D()(X)
    X = Flatten()(X)

    # Output
    X = Dense(units=num_classes, activation='softmax')(X)

    model = Model(inputs=input, outputs=X)

    return model


In [59]:
model = ResNet_Cifar10(input_shape=(32, 32, 3), num_classes=10)
model.summary()

Model: "model_19"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_24 (InputLayer)       [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 conv2d_456 (Conv2D)         (None, 32, 32, 64)           1792      ['input_24[0][0]']            
                                                                                                  
 batch_normalization_393 (B  (None, 32, 32, 64)           256       ['conv2d_456[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activation_441 (Activation  (None, 32, 32, 64)           0         ['batch_normalization_3

# 5. Generalize ResNet

In [65]:
def ResNet(input_shape=(224, 224, 3), num_classes=1000, stages=[3, 4, 6, 3], filters=[64, 256, 512, 1024, 2048]):
    input = Input(input_shape)

    # Stage 01
    X = Conv2D(filters=filters[0], kernel_size=7, strides=2, activation='relu')(input)
    X = BatchNormalization()(X)
    X = MaxPooling2D(3, strides=2, padding='same')(X)

    # Stage 02 -> 05
    for stage, filter in zip(stages, filters[1:]):
        filters_in_stage = [int(filter / 4), int(filter / 4), filter]
        for i in range(stage):
            if i == 0:
                X = convolutional_block(X, filters=filters_in_stage)
            else:
                X = identity_block(X, filters=filters_in_stage)
    
    X = GlobalAveragePooling2D()(X)

    # Output layer
    X = Flatten()(X)
    X = Dense(units=num_classes, activation='softmax')(X)

    model = Model(inputs=input, outputs=X)

    return model


In [66]:
# ResNet50
model50 = ResNet(input_shape=(224, 224, 3), num_classes=1000, 
                 stages=[3,4,6,3], filters=[64,256,512,1024,2048])
model50.summary()

Model: "model_20"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_26 (InputLayer)       [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d_514 (Conv2D)         (None, 109, 109, 64)         9472      ['input_26[0][0]']            
                                                                                                  
 batch_normalization_438 (B  (None, 109, 109, 64)         256       ['conv2d_514[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 max_pooling2d_10 (MaxPooli  (None, 55, 55, 64)           0         ['batch_normalization_4

In [68]:
# ResNet101
model101 = ResNet(input_shape=(224, 224, 3), num_classes=1000, stages=[3,4,23,3], filters=[64,256,512,1024,2048])
model101.summary()

Model: "model_21"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_27 (InputLayer)       [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d_579 (Conv2D)         (None, 109, 109, 64)         9472      ['input_27[0][0]']            
                                                                                                  
 batch_normalization_491 (B  (None, 109, 109, 64)         256       ['conv2d_579[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 max_pooling2d_11 (MaxPooli  (None, 55, 55, 64)           0         ['batch_normalization_4

In [70]:
# ResNet152
model152 = ResNet(input_shape=(224, 224, 3), num_classes=1000, stages=[3,8,36,3], filters=[64,256,512,1024,2048])
model152.summary()

Model: "model_22"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_28 (InputLayer)       [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d_712 (Conv2D)         (None, 109, 109, 64)         9472      ['input_28[0][0]']            
                                                                                                  
 batch_normalization_595 (B  (None, 109, 109, 64)         256       ['conv2d_712[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 max_pooling2d_12 (MaxPooli  (None, 55, 55, 64)           0         ['batch_normalization_5

# 6. Train ResNet50 on CIFAR10

In [71]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

# Load CIFAR10 dataset
(cifar_X_train, cifar_y_train), (cifar_X_test, cifar_y_test) = cifar10.load_data()

# Normalizing between 0 and 1
cifar_X_train = cifar_X_train / 255.0
cifar_X_test = cifar_X_test / 255.0
#plt.imshow(x_train[0])


# Find num_of_classes and converting to one-hot
num_classes = len(np.unique(cifar_y_train))
cifar_Y_train = to_categorical(cifar_y_train, num_classes)
cifar_Y_test = to_categorical(cifar_y_test, num_classes)

# Split to validation set from train set
cifar_X_train, cifar_X_val = cifar_X_train[5000:, ...], cifar_X_train[:5000, ...]
cifar_Y_train, cifar_Y_val = cifar_Y_train[5000:, ...], cifar_Y_train[:5000, ...]

print(f"X_train.shape: {cifar_X_train.shape}")
print(f"Y_train.shape: {cifar_Y_train.shape}")
print(f"X_val.shape: {cifar_X_val.shape}")
print(f"Y_val.shape: {cifar_Y_val.shape}")
print(f"X_test.shape: {cifar_X_test.shape}")
print(f"Y_test.shape: {cifar_Y_test.shape}")

  8503296/170498071 [>.............................] - ETA: 1:39

KeyboardInterrupt: 

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

# ResNet50
resnet50_cifar = ResNet(cifar_X_train.shape[1:], num_classes,
                 stages=[3,4,6,3], filters=[64,256,512,1024,2048])

resnet50_cifar.compile(optimizer='adam',
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath='cifar_resnet.weights.best.hdf5',
                               verbose=1,
                               save_best_only=True)

callbacks = [checkpointer]

hist = resnet50_cifar.fit(cifar_X_train, cifar_Y_train,
                        validation_data=(cifar_X_val, cifar_Y_val),
                        batch_size=1000, epochs=30,
                        callbacks=callbacks,
                        verbose=1, shuffle=True)

In [ ]:
import matplotlib.pyplot as plt

train_loss, val_loss = hist.history['loss'], hist.history['val_loss']
train_acc, val_acc = hist.history['accuracy'], hist.history['val_accuracy']

# Create a figure with two axes
fig, axs = plt.subplots(figsize=(8,3), nrows=1, ncols=2)

# Plot loss on the left axis
num_epochs = np.arange(len(train_loss))
axs[0].plot(num_epochs, train_loss)
axs[0].plot(num_epochs, val_loss)
axs[0].set_ylabel('loss')
axs[0].set_xlabel('epoch')
axs[0].set_title('model loss on CIFAR10')
axs[0].legend(['train', 'validation'], loc='upper right')

axs[1].plot(num_epochs, train_acc)
axs[1].plot(num_epochs, val_acc)
axs[1].set_ylabel('acc')
axs[1].set_xlabel('epoch')
axs[1].set_title('model accuracy on CIFAR10')
axs[1].legend(['train', 'validation'], loc='upper right')

plt.tight_layout()
plt.show()

In [ ]:
# Evaluate
resnet50_cifar.load_weights('./cifar_resnet.weights.best.hdf5')
score = resnet50_cifar.evaluate(cifar_X_test, cifar_Y_test, verbose=0)

print(f"Test accuracy: {score[1] * 100}%")